In [4]:
import pandas as pd
import psycopg2 as psy
from sqlalchemy import create_engine

In [6]:
orders_df=pd.read_csv('/Users/sriharsha/orders.csv')
orders_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [8]:
orders_df.drop(columns='eval_set',inplace=True,axis=1)

In [10]:
products_df=pd.read_csv('products.csv')
products_df.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [12]:
order_prod_df=pd.read_csv('order_products.csv').sample(100)
order_prod_df.head()

,order_id,product_id,add_to_cart_order,reordered
2218189,233923,37094,10,1
15557621,1641472,4362,10,0
19740756,2082005,15237,1,1
12567301,1326346,23695,8,1
21896897,2309623,42265,10,1


In [14]:
aisles_df=pd.read_csv('aisles.csv')
aisles_df.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [16]:
departments_df=pd.read_csv('departments.csv')
departments_df.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [40]:
try:
    conn=psy.connect(dbname='ecom_instamart',user='postgres',password='#',port='5432')
    cur=conn.cursor()
except Exception as e:
    print(e)

In [20]:
engine=create_engine('postgresql+psycopg2://postgres:#@localhost/ecom_instamart')

In [126]:
cur.execute('''
create table orders(
    order_id integer primary key,
    user_id integer,
    order_number integer,
    order_dow integer,
    order_hour_of_day integer,
    days_since_prior_order integer    
)   '''
)

In [108]:
cur.execute('''
create table aisles(
    aisle_id integer primary key,
    aisle varchar(50))
''')

In [110]:
cur.execute('''
create table departments(
    department_id integer primary key,
    department varchar(50))
''')

In [132]:
conn.commit()

In [120]:
cur.execute('''
create table products(
    product_id integer primary key,
    product_name varchar(255),
    aisle_id integer,
    department_id integer,
    FOREIGN KEY (aisle_id) REFERENCES aisles (aisle_id),
    FOREIGN KEY (department_id) REFERENCES departments (department_id)
    )
''')

In [33]:
cur.execute('''
create table order_products(
    order_id integer,
    product_id integer  ,
    add_to_cart_order varchar(255),
    reordered boolean,
    PRIMARY KEY (order_id,product_id),
    FOREIGN KEY (order_id) REFERENCES orders (order_id),
    FOREIGN KEY (product_id) REFERENCES products (product_id)
    )
''')

In [184]:
aisles_df.to_sql('aisles',con=engine,if_exists='append',index=False)

134

In [44]:
conn.commit()

In [188]:
departments_df.to_sql('departments',con=engine,if_exists='append',index=False)

21

In [46]:
order_prod_df.to_sql('order_products',con=engine,if_exists='append',index=False)

100

In [23]:
products_df.to_sql('products',con=engine,if_exists='append',index=False)

688

In [63]:
orders_df.to_sql('orders',con=engine,if_exists='append',index=False)

83

In [41]:
orders_df.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


In [ ]:
order_products_df.head()

In [42]:
cur.execute(
'''
alter table order_products
alter column reordered type integer using reordered::integer
'''
)